<a href="https://colab.research.google.com/github/wczubal1/BloombergTask/blob/main/BloombergTaskWitoldCzubala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from statistics import NormalDist

In [24]:
!pip install scikit-learn

In [26]:
import sklearn

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#Uploading corporate PD datset
data_root = "https://raw.githubusercontent.com/wczubal1/BloombergTask/main/"
df_credit = pd.read_csv(data_root + "df_credit.csv")

In [11]:
df_credit

,Instrument,CompanyName,BusinessSector,CompanyMarketCap,CreditCombinedPD,ReturnOnAssets,TotalDebttoCommonEquity,PriceMomentumT12MVolatility,randNumCol,Default
1,FCNCA.OQ,First Citizens BancShares Inc (Delaware),Banking & Investment Services,2.005774e+10,0.251288,1.010,0.92905,24.651832,0.635039,False
4,DDD.N,3D Systems Corp,Technology Equipment,6.925227e+08,0.459605,-1.960,0.60469,21.900110,0.899381,False
5,MMM.N,3M Co,Consumer Goods Conglomerates,5.971652e+10,0.139720,12.259,1.08851,8.019977,0.905377,False
8,EGHT.OQ,8x8 Inc,Telecommunications Services,4.287940e+08,0.317889,4.400,4.92209,24.693342,0.823953,False
9,AOS.N,A O Smith Corp,Industrial Goods,1.206271e+10,0.029621,14.359,0.19712,4.928321,0.095575,False
...,...,...,...,...,...,...,...,...,...,...
2321,ZIP.N,Ziprecruiter Inc,Industrial & Commercial Services,1.407385e+09,0.262709,11.050,18.92240,22.864605,0.740800,False
2322,ZTS.N,Zoetis Inc,Pharmaceuticals & Medical Research,9.133608e+10,0.053257,15.939,1.79432,6.464926,0.085514,False
2324,ZI.OQ,Zoominfo Technologies Inc,Software & IT Services,6.127379e+09,0.216172,5.200,0.54393,18.089863,0.992130,False
2325,ZS.OQ,Zscaler Inc,Software & IT Services,3.423091e+10,0.049608,8.650,1.56412,16.097909,0.255266,False


In [4]:
#Count of rows with missing values
print (df_credit.isna().any(axis=1).sum())

802


In [5]:
#Removing missing values
df_credit=df_credit.dropna()

In [6]:
df_credit.loc[:,'randNumCol'] = np.random.uniform(0, 1, df_credit.shape[0])

<ipython-input-6-d4b71def6516>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit.loc[:,'randNumCol'] = np.random.uniform(0, 1, df_credit.shape[0])


In [9]:
df_credit['CreditCombinedPD'].describe()
#df_credit['randNumCol'].describe()

count    1527.000000
mean        0.283032
std         0.493445
min         0.006694
25%         0.074972
50%         0.140495
75%         0.269909
max         5.972882
Name: CreditCombinedPD, dtype: float64

In [10]:
#simulating default
df_credit.loc[:,'Default'] = df_credit['CreditCombinedPD']/100>df_credit['randNumCol']

<ipython-input-10-7c77fd14bf5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit.loc[:,'Default'] = df_credit['CreditCombinedPD']/100>df_credit['randNumCol']


In [17]:
sum(df_credit['Default'])
df_credit.loc[df_credit['Default']==True]
df_credit.loc[df_credit['Default']==True,'BusinessSector'].describe()

count               9
unique              6
top       Real Estate
freq                3
Name: BusinessSector, dtype: object

In [44]:
#Sector variables
df_credit.groupby('BusinessSector').size()
df_credit=pd.concat([df_credit, pd.get_dummies(df_credit['BusinessSector'],drop_first=True)],axis=1)

In [49]:
df_estimation= df_credit[df_credit.columns.difference(['Instrument', 'CompanyName', 'BusinessSector', 'CompanyMarketCap', 'CreditCombinedPD', 'randNumCol'])]

In [52]:
df_estimation.columns
#df_credit

Index(['Automobiles & Auto Parts', 'Banking & Investment Services',
       'Chemicals', 'Consumer Goods Conglomerates',
       'Cyclical Consumer Products', 'Cyclical Consumer Services', 'Default',
       'Energy - Fossil Fuels',
       'Financial Technology (Fintech) & Infrastructure', 'Food & Beverages',
       'Food & Drug Retailing', 'Healthcare Services & Equipment',
       'Industrial & Commercial Services', 'Industrial Goods', 'Insurance',
       'Mineral Resources', 'Personal & Household Products & Services',
       'Pharmaceuticals & Medical Research', 'PriceMomentumT12MVolatility',
       'Real Estate', 'Renewable Energy', 'Retailers', 'ReturnOnAssets',
       'Software & IT Services', 'Technology Equipment',
       'Telecommunications Services', 'TotalDebttoCommonEquity',
       'Transportation', 'Utilities'],
      dtype='object')

In [63]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_estimation.drop(['Default'],axis=1), df_estimation['Default'], test_size=0.25)

# Create the logistic regression model
model = LogisticRegression(penalty='l2', C=0.01)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

In [64]:
# Evaluate the model
accuracy = model.score(X_test, y_test)
print('Accuracy:', accuracy)
accuracy = model.score(X_train, y_train)
print('Accuracy:', accuracy)

Accuracy: 0.9921465968586387
Accuracy: 0.9956331877729258


In [65]:
model.coef_
#confusion_matrix(y_test, model.predict(X_test))
#confusion_matrix(y_train, model.predict(X_train))

array([[ 0.00942934, -0.00855378, -0.00089824, -0.00012389, -0.00183938,
        -0.00115629, -0.00212892, -0.00028169, -0.00126358, -0.00029112,
         0.01576723, -0.00226028, -0.00301397, -0.00122794, -0.00057146,
        -0.00067766, -0.00252832,  0.00046395,  0.01616338, -0.00051189,
        -0.00177813,  0.00164291, -0.00564125, -0.00311987, -0.00046285,
         0.03544886, -0.00120439, -0.00134836]])

In [66]:
print(classification_report(y_train, model.predict(X_train)))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00      1139
        True       1.00      0.17      0.29         6

    accuracy                           1.00      1145
   macro avg       1.00      0.58      0.64      1145
weighted avg       1.00      1.00      0.99      1145



In [40]:
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}